# 보이스 챗봇 만들기

### 🍉Watermelon: 물 흐르듯 원하는 노래를 찾아드립니다 🕵️
1. 사용자 입력을 음성으로 받는다.
2. STT를 적용하여 텍스트로 변환한다.
3. 변환된 텍스트를 입력으로 하여 프롬프트 엔지니어링을 해 api 요청을 보낸다.
4. 반환받은 응답을 TTS를 적용하여 음성으로 재생한다.
(+) 2에서 입력된 텍스트와 4에서 반환된 응답을 채팅 내역 보듯이  
(카카오톡 대화처럼) 현출되도록 출력한다.

In [1]:
!pip install openai
!pip install SpeechRecognition
!pip install pydub pyaudio
!pip install pyttsx3

In [2]:
from dotenv import load_dotenv
import os
import speech_recognition as sr
import tempfile
import threading
import re
import pyttsx3
from openai import OpenAI
from playsound import playsound

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
def music_find(temperature=0.3):
    client = OpenAI(api_key=OPENAI_API_KEY)
    recognizer = sr.Recognizer()

    system_instruction = """
    너는 사용자가 원하는 음악을 친절하게 찾아주는 챗봇이야.
    사용자가 말하는 가수 이름, 노래 제목, 가사 일부, 분위기, 감정 키워드, 날씨 등을 기반으로 노래를 추천해줘.

    ### 지시 사항 ###
    1. 사용자가 애매한 정보를 말해도 가장 정확히 유추하여 노래를 찾을 것
    2. 여러 후보가 있을 경우, 6곡 이내로 압축해서 추천할 것
    3. 사용자가 '찾았다'라고 말하면 대화를 중단할 것
    4. 추론이 완료된 즉시 가장 빠른 속도로 추론하여 결과를 도출할 것
    5. 부정확한 답변(가상의 가수나 노래 지어내기, 오류 정보 전달)은 금지할 것
"""

    # 대화 기록 초기화
    messages = [
        {"role": "system", "content": system_instruction}
    ]

    # 챗봇 응답 TTS 백그라운드 재생 함수: 별도로 플레이어 로드 없이 바로 음성이 나오도록
    # 속도, 볼륨 조절
    def play_tts(text):
        engine = pyttsx3.init()         # 루프 안에서 새 엔진 생성
        engine.setProperty("rate", 180) # 속도 조절
        engine.setProperty("volume", 1) # 볼륨 조절
        engine.say(text)
        engine.runAndWait()
        engine.stop()
        
    print("🎵 음악 챗봇이 시작되었습니다! 노래를 찾았다면 '찾았다'라고 말해주세요.")
    print("🍉어떤 노래를 찾고 싶은가요?🎤\n", flush=True)       #flush=True: 화면에 바로 출력될 수 있게 함

    mic = sr.Microphone()
    while True:
        # STT: 사용자의 음성(Sound) -> 텍스트화(Text)
        with mic as source:
            audio = recognizer.listen(source)

        # 예외처리: 입력이 들어오지 않을 경우
        try:
            user_text = recognizer.recognize_google(audio, language="ko-KR")
            print(f"🕵️ 사용자: {user_text}", flush=True)
        except Exception as e:
            print("음성 인식 오류:", e, flush=True)
            continue

        # 챗봇과의 대화 종료 조건
        if user_text.strip() == "찾았다":
            print("필요하면 또 찾아와요!😊", flush=True)
            break

        # 이전 내용을 기억할 수 있도록 대화 기록에 추가
        messages.append({"role": "user", "content": user_text})

        # 챗봇 호출
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=temperature,
            max_tokens=2048
        )
        answer = response.choices[0].message.content

        # 응답에 *, - 같은 표시가 나와서 tts가 그대로 읽음 -> answer 전처리 진행
        answer = re.sub(r"[*-]", "", answer)

        print(f"🍉Watermelon: {answer}", flush=True)

        # GPT 응답 기억 및 기록
        messages.append({"role": "assistant", "content": answer})

        # 🎧 음성이 끝날 때까지 기다림
        play_tts(answer)

    return messages


In [4]:
music_find()

🎵 음악 챗봇이 시작되었습니다! 노래를 찾았다면 '찾았다'라고 말해주세요.
🍉어떤 노래를 찾고 싶은가요?🎤

🕵️ 사용자: 비 오는 날에 어울리는 노래 추천해 줘
🍉Watermelon: 비 오는 날에 어울리는 노래로는 다음과 같은 곡들이 있습니다:

1. 폴킴  "비"
2. 에픽하이  "우산 (Feat. 윤하)"
3. 헤이즈  "비도 오고 그래서 (Feat. 신용재)"
4. 아이유  "Rain Drop"
5. 정승환  "비가 온다"
6. 김현식  "비처럼 음악처럼"

이 곡들은 비 오는 날의 감성을 잘 담고 있어 듣기 좋습니다.
🕵️ 사용자: 신나는 노래로 추천해 줄 수 없어
🍉Watermelon: 신나는 분위기의 노래로는 다음과 같은 곡들이 있습니다:

1. 싸이 "강남스타일"
2. 방탄소년단 "Dynamite"
3. 트와이스 "Dance The Night Away"
4. 레드벨벳 "Red Flavor"
5. 블랙핑크 "붐바야"
6. 마마무 "HIP"

이 곡들은 에너지가 넘치고 신나는 분위기를 느낄 수 있어 기분 전환에 좋습니다.
🕵️ 사용자: 트와이스 노래 추천해 줄 수 있어
🍉Watermelon: 트와이스의 인기 있는 노래들 중 몇 곡을 추천해드릴게요:

1. "CHEER UP"
2. "TT"
3. "LIKEY"
4. "FANCY"
5. "Feel Special"
6. "What is Love?"

이 곡들은 트와이스의 매력을 잘 느낄 수 있는 곡들입니다.
🕵️ 사용자: 뮤직비디오 컨셉이 할로윈인형
🍉Watermelon: 트와이스의 "TT" 뮤직비디오는 할로윈과 인형 컨셉을 가지고 있습니다. 이 뮤직비디오는 멤버들이 다양한 할로윈 코스튬을 입고 등장하며, 귀엽고 신비로운 분위기를 연출합니다. "TT"는 트와이스의 대표적인 곡 중 하나로 많은 사랑을 받았습니다.
🕵️ 사용자: 빠졌다
🍉Watermelon: 좋아요! "TT"는 정말 매력적인 곡이죠. 다른 추천이나 도움이 필요하시면 언제든지 말씀해 주세요!
🕵️ 사용자: 빠졌다
🍉Watermelon: 알겠습

[{'role': 'system',
  'content': "\n    너는 사용자가 원하는 음악을 친절하게 찾아주는 챗봇이야.\n    사용자가 말하는 가수 이름, 노래 제목, 가사 일부, 분위기, 감정 키워드, 날씨 등을 기반으로 노래를 추천해줘.\n\n    ### 지시 사항 ###\n    1. 사용자가 애매한 정보를 말해도 가장 정확히 유추하여 노래를 찾을 것\n    2. 여러 후보가 있을 경우, 6곡 이내로 압축해서 추천할 것\n    3. 사용자가 '찾았다'라고 말하면 대화를 중단할 것\n    4. 추론이 완료된 즉시 가장 빠른 속도로 추론하여 결과를 도출할 것\n    5. 부정확한 답변(가상의 가수나 노래 지어내기, 오류 정보 전달)은 금지할 것\n"},
 {'role': 'user', 'content': '비 오는 날에 어울리는 노래 추천해 줘'},
 {'role': 'assistant',
  'content': '비 오는 날에 어울리는 노래로는 다음과 같은 곡들이 있습니다:\n\n1. 폴킴  "비"\n2. 에픽하이  "우산 (Feat. 윤하)"\n3. 헤이즈  "비도 오고 그래서 (Feat. 신용재)"\n4. 아이유  "Rain Drop"\n5. 정승환  "비가 온다"\n6. 김현식  "비처럼 음악처럼"\n\n이 곡들은 비 오는 날의 감성을 잘 담고 있어 듣기 좋습니다.'},
 {'role': 'user', 'content': '신나는 노래로 추천해 줄 수 없어'},
 {'role': 'assistant',
  'content': '신나는 분위기의 노래로는 다음과 같은 곡들이 있습니다:\n\n1. 싸이 "강남스타일"\n2. 방탄소년단 "Dynamite"\n3. 트와이스 "Dance The Night Away"\n4. 레드벨벳 "Red Flavor"\n5. 블랙핑크 "붐바야"\n6. 마마무 "HIP"\n\n이 곡들은 에너지가 넘치고 신나는 분위기를 느낄 수 있어 기분 전환에 좋습니다.'},
 {'role': 'user', 'content'